In [1]:
import pandas as pd
import tkinter as tk
from tkinter import ttk  # Import ttk module for Treeview
from tkinter import StringVar
import requests
from io import StringIO

df = None  # Define df at a higher scope
tree = None  # Define tree at a higher scope
colunas_selecionadas = None  # Defina colunas_selecionadas como uma variável global

def determinar_delimitador(content):
    delimiters = [',', ';', '\t']  # Add other delimiters as needed
    for delimiter in delimiters:
        if delimiter in content:
            return delimiter
    return ','  # Default to comma if no delimiter is found

def carregar_arquivo():
    global df, tree, colunas_selecionadas
    url = entrada_url.get()

    # Faça o download do conteúdo do arquivo da web
    response = requests.get(url)
    
    # Verifique se a solicitação foi bem-sucedida
    if response.status_code != 200:
        print(f"Erro ao carregar o arquivo da web. Código de status: {response.status_code}")
        return

    # Obtenha o conteúdo do arquivo como texto
    content = response.text

    # Tente determinar o delimitador com base nas primeiras linhas do conteúdo
    delimitador = determinar_delimitador(content)

    # Leitura do conteúdo como DataFrame com pandas
    try:
        df = pd.read_csv(StringIO(content), sep=delimitador, on_bad_lines='skip', dtype=str)
    except Exception as e:
        print(f"Erro ao carregar o arquivo: {str(e)}")
        return

    # Obtenha o nome das colunas do DataFrame
    colunas = df.columns.tolist()

    # Se o usuário não especificar nenhuma coluna, selecione automaticamente as 4 primeiras colunas
    if colunas_selecionadas[0] == '':
        colunas_selecionadas = colunas[:4]
    else:
        colunas_selecionadas = [col.strip() for col in colunas_selecionadas]
        
    # Create Combobox widgets for filtering by column names
    filter_comboboxes = {}
    for col in colunas_selecionadas:
        filtro = StringVar()
        filtro.set("Todas")  # Define o valor padrão como "Todas"
        combobox = ttk.Combobox(frame_filtros, textvariable=filtro)
        combobox['values'] = ["Todas"] + list(df[col].unique())
        combobox.grid(row=0, column=colunas_selecionadas.index(col), sticky="w")
        filter_comboboxes[col] = filtro

    # Create a filter button
    botao_filtrar = tk.Button(frame_filtros, text="Filtrar", command=filtar_tabela)
    botao_filtrar.grid(row=1, column=0, columnspan=len(colunas_selecionadas), pady=5)

    # Create a new Treeview widget with columns and headings
    columns = colunas_selecionadas

    # Create a frame for the Treeview with scrollbars
    tree_frame = ttk.Frame(root)
    tree_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")
    tree_frame.grid_rowconfigure(0, weight=1)
    tree_frame.grid_columnconfigure(0, weight=1)

    canvas = tk.Canvas(tree_frame)
    canvas.grid(row=0, column=0, sticky="nsew")

    vsb = ttk.Scrollbar(tree_frame, orient="vertical", command=canvas.yview)
    vsb.grid(row=0, column=1, sticky="ns")
    canvas.configure(yscrollcommand=vsb.set)

    tree = ttk.Treeview(canvas, columns=columns, show="headings")
    tree.grid(row=0, column=0, sticky="nsew")

    hsb = ttk.Scrollbar(tree_frame, orient="horizontal", command=canvas.xview)
    hsb.grid(row=1, column=0, sticky="ew")
    canvas.configure(xscrollcommand=hsb.set)

    canvas.create_window((0, 0), window=tree, anchor="nw")
    canvas.update_idletasks()
    canvas.configure(scrollregion=canvas.bbox("all"))

    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=100)

    mostrar_tabela(tree)

def mostrar_tabela(tree):
    if df is not None:
        for i in tree.get_children():
            tree.delete(i)
        for index, row in df.iterrows():
            values = [row[col] for col in df.columns]
            tree.insert("", "end", values=values)

def filtar_tabela():
    if df is not None:
        selected_filters = {col: filtro.get() for col, filtro in filter_comboboxes.items()}
        mask = pd.Series([True] * len(df))
        for col, value in selected_filters.items():
            if value != "Todas" and col in df.columns:
                mask &= (df[col] == value)
        for i in tree.get_children():
            tree.delete(i)
        for index, row in df[mask].iterrows():
            values = [row[col] for col in df.columns]
            tree.insert("", "end", values=values)

root = tk.Tk()
root.title("Leitor de CSV Online")

frame_url = tk.Frame(root)
frame_url.grid(row=0, column=0, padx=10, pady=5, sticky="w")
label_url = tk.Label(frame_url, text="URL do CSV:")
label_url.grid(row=0, column=0)
entrada_url = tk.Entry(frame_url)
entrada_url.grid(row=0, column=1)

frame_colunas = tk.Frame(root)
frame_colunas.grid(row=1, column=0, padx=10, pady=5, sticky="w")
label_colunas = tk.Label(frame_colunas, text="Colunas (separadas por vírgula):")
label_colunas.grid(row=0, column=0)
entrada_colunas = tk.Entry(frame_colunas)
entrada_colunas.grid(row=0, column=1)

frame_filtros = tk.Frame(root)
frame_filtros.grid(row=2, column=0, columnspan=2, padx=10, pady=5, sticky="w")

botao_carregar = tk.Button(root, text="Carregar Arquivo", command=carregar_arquivo)
botao_carregar.grid(row=3, column=0, columnspan=2, pady=10)

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\Users\Eduarda\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Eduarda\AppData\Local\Temp\ipykernel_12464\3243597683.py", line 28, in carregar_arquivo
    delimitador = determinar_delimitador(content)
                  ^^^^^^^^^^^^^^^^^^^^^^
NameError: name 'determinar_delimitador' is not defined
